# TME 5 : Localisation de pattern (motifs) dans le genome

Nous allons implémenter plusieurs algorithmes afin de localiser un motif d'intérêt dans un génome.

### A -  Index of fixed length words
3\. Nous allons implémenter l'algorithme "Index of fixed length words". Faire une fonction pour indexer les positions d'occurrences de tous les mots de taille k dans un texte (génome), la fonction doit renvoyer un dictionnaire ou les clés sont les mots et les valeurs les positions dans le texte où se trouve les mots.

| nt | proba |
| :-: | :-: |
| A | ... |
| C | ... |
| G | ... |
| T | ... |

In [136]:
def indexTable(m, sequence):
    """
    Indexer les positions d'occurrences de tous les mots de taille k dans une sequence
    entrée m : taille du mot à chercher dans le motif m <= k
    entrée sequence : chaine de caractère représentant une sequence d'ADN
    sortie indexes : dictionaire où les clés sont les mots et les valeurs les positions dans la sequence
    """
    indexes  = {}
    for i in range(len(sequence)) :
        if i + m > len(sequence) : break
        
        aux = sequence[i:i+m] 
        if aux in indexes : indexes[aux].append(i)
        else : indexes[aux] = [i]

    return indexes

inds = indexTable(2, "abcdabda") #{'ab': [0, 4], 'bc': [1], 'cd': [2], 'da': [3, 6], 'bd': [5]}
print (inds)

{'ab': [0, 4], 'bc': [1], 'cd': [2], 'da': [3, 6], 'bd': [5]}


4\. Faire une fonction pour chercher toutes les occurrences d'un motif dans un génome utilisant la table des indexes de la question précédente. Nous allons permettre un nombre maximum de variations. Pour cela, implémenter ou re-utiliser la fonction distance de hamming

In [137]:
def hamming(seq1, seq2):
    cpt = 0
    limit = len(seq1)
    if len(seq1) >= len(seq2) : limit = len(seq2)
    for i in range(limit) :
        if seq1[i] != seq2[i] : cpt += 1
    return cpt

def chercherWithIndexTable(m, table, sequence, motif, maxVar):
    """
    chercher les positions d'un motif dans une séquence en admettant au maximum maxVar variations
    entrée m : taille du mot à chercher dans le motif m <= k, le meme utilise pour indexer les sequences
    entrée table : dictionaire où les clés sont les mots et les valeurs les positions dans la sequence
    entrée sequence : chaine de caractère représentant une sequence d'ADN
    entrée motif : chaine de caractère représentant le motif à chercher
    entrée maxVar : le maximum variations entre le motif et un mot de taille k dans la sequence
    sortie motifPos : dictionnaire où les clés sont les motifs trouvé et les valeurs leurs positions dans la sequence.
    """
    k = len(motif)
    motif = motif.lower()
    motifPos = dict()

    for cand,pos in table.items() :
        if cand.lower() in motif :
            aux_dict = dict()
            for p in pos :
                tmp = sequence[p:p+k].lower()
                if hamming(motif,tmp) < maxVar : 
                    if tmp in aux_dict : aux_dict[tmp].append(p)
                    else : aux_dict[tmp] = [p]
            if len(aux_dict) > 0 :
                for x,lst in aux_dict.items() :
                    motifPos[x] = lst

    return motifPos


m = 3; motif =  "TAGTT"; maxVar = 2
inds = indexTable(m, "ACGAAATAGTTAGAA")
print (inds)
#{'acg': [0], 'cga': [1], 'gaa': [2, 12], 'aaa': [3], 'aat': [4], 'ata': [5], 'tag': [6, 10], 'agt': [7], 'gtt': [8], 'tta': [9], 'aga': [11]}
motifPos = chercherWithIndexTable(m, inds, 'ACGAAATAGTTAGAA', "TAGTT", maxVar)
print (motifPos)
#{'tagtt': [6]}

{'ACG': [0], 'CGA': [1], 'GAA': [2, 12], 'AAA': [3], 'AAT': [4], 'ATA': [5], 'TAG': [6, 10], 'AGT': [7], 'GTT': [8], 'TTA': [9], 'AGA': [11]}
{'tagtt': [6]}


5\. Tester l'algorithme **"Index of fixed length words"** sur vos données de chipSeq. Pour cela développer une fonction pour chercher un motif d'intérêt sur chaque séquence du fichier Peaks.
Voici les motifs que vous devez trouver selon le fichier de Peaks:

 - Peaks 1 Atf1 (TGCACCC / GGGTGCA)
 - Peaks 2 Atm1 (CAGCAAaaa / tttTTGCTG)
 - Peaks 3 Hap4 (CCAAT / ATTGG)
 - Peaks 4 Mac1 (GAGCAAA / TTTGCTC)
 - Peaks 5 Mac1 (GAGCAAA / TTTGCTC)
 - Peaks 6 Pdr1 (CCACGGA / TCCGTGG)
 - Peaks 7 Pdr1 (CCACGGA / TCCGTGG)

In [138]:
import time
import itertools

k = 7
genome = "C_glabrata_1000bp_only.fasta"
peaks = "Sequence_by_Peaks_1.fasta" ### modifier avec votre fichier !!!
motif = 'gggtgca' # Le motif à localiser pour votre fichier, par example "tgcaccc"

# Fonction modifié
def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie sequences: liste contenant toutes les sequences du fichier
    """

    sequence = ""
    sequences_list = []
    prev_header = ""
    header = ""

    for line in open(fastaFileName):
        string = line.strip()
        if string[0] != ">":
            if prev_header != header:
                # print(header) # dans le cas ou on veut garde cet info
                prev_header = header
            sequence = sequence + string
        else:
            header = string
            if sequence != "":
                sequences_list.append(sequence)
                sequence = ""

    sequences_list.append(sequence)
    return sequences_list

sequencesPeaks = readFasta(peaks)
sequences1000 = readFasta(genome)


def findMotifData(sequences, motif, m, maxVar):
    """
    chercher les positions d'un motif dans un ensemble de séquence d'ADN en admettant un maximum de variations
    entrée m : taille du mot à chercher dans le motif m <= k, le meme utilise pour indexer les sequences
    entrée sequences : list contenant les sequence d'ADN
    entrée motif : chaine de caractère représentant le motif à chercher
    entrée maxVar : le maximum variations entre le motif et un mot de taille k dans la sequence
    sortie posList : list contenant les positions dans les sequences ou se trouve le motif.
    """
    #count = 0
    posList = []
    posDict_seq = dict()
    i = 0
    while i < len(sequences) :
        idx = indexTable(m, sequences[i])
        posDict = chercherWithIndexTable(m, idx, sequences[i], motif, maxVar)
        if len(posDict) > 0 : posDict_seq[i] = posDict

        for _,pos in posDict.items() : 
            posList += pos

        i += 1

    print(posDict_seq)
    return posList



In [139]:
posList = findMotifData(sequencesPeaks, motif, 4, 1)
print(posList)
print(len(posList))


{0: {'gggtgca': [347]}, 1: {'gggtgca': [307]}, 4: {'gggtgca': [162]}, 11: {'gggtgca': [267]}, 13: {'gggtgca': [74]}, 29: {'gggtgca': [117]}, 38: {'gggtgca': [349]}, 40: {'gggtgca': [791]}, 46: {'gggtgca': [288]}, 53: {'gggtgca': [704]}, 55: {'gggtgca': [246]}, 77: {'gggtgca': [164]}, 89: {'gggtgca': [92]}, 93: {'gggtgca': [261]}, 101: {'gggtgca': [73]}, 102: {'gggtgca': [193]}, 112: {'gggtgca': [34]}}
[347, 307, 162, 267, 74, 117, 349, 791, 288, 704, 246, 164, 92, 261, 73, 193, 34]
17


6\. Récupérer les motifs et générer un LOGO à l'aide de l'outil WEB-LOGO https://weblogo.berkeley.edu/logo.cgi

### B -  Matrice de fréquences
7\.Nous allons implémenter la recherche de motif par matrice de fréquence. Nous allons utiliser les matrices déjà fabriquées que vous pouvez télécharger sur le site http://jaspar.genereg.net/.
Une fois que vous avez chargé votre matrice de fréquence, vous devez la transformer en matrice de probabilité en la normalisant, diviser chaque cellule par la somme de sa colonne.

In [140]:
import numpy as np

nuc = ['A', 'C', 'G', 'T']
q = 4

#Matrice du motif Atf1, attention la matrice peut avoir plus de colonnes que le motif décrit précédemment,
#ajuster le k ou effacer les colonnes qui ne vous intéressent pas.

matFreq = np.array([[136,716,145,34,160,10,961,8,25,19],
[78,25,31,47,10,966,9,972,955,740],
[142,31,288,39,812,11,15,9,2,13],
[641,226,535,878,16,10,14,10,16,225]])

display(matFreq)

def computing_pwm(M, cols):
    """
    Calcul la matrice de poids position à partir de la matrice de frequence
    entrée M : matrice de frequence
    sortie PWM : matrice de probabilites ou poids position
    """
    PWM = M+1
    sum_col = np.sum(PWM,axis=0)

    return PWM/sum_col

k = 10
PWM = computing_pwm(matFreq, k)
display(PWM)


array([[136, 716, 145,  34, 160,  10, 961,   8,  25,  19],
       [ 78,  25,  31,  47,  10, 966,   9, 972, 955, 740],
       [142,  31, 288,  39, 812,  11,  15,   9,   2,  13],
       [641, 226, 535, 878,  16,  10,  14,  10,  16, 225]])

array([[0.13686314, 0.71556886, 0.14556331, 0.03493014, 0.16067864,
        0.01098901, 0.95912263, 0.00897308, 0.0259481 , 0.01998002],
       [0.07892108, 0.0259481 , 0.03190429, 0.04790419, 0.01097804,
        0.96603397, 0.00997009, 0.97008973, 0.95409182, 0.74025974],
       [0.14285714, 0.03193613, 0.28813559, 0.03992016, 0.81137725,
        0.01198801, 0.01595214, 0.00997009, 0.00299401, 0.01398601],
       [0.64135864, 0.22654691, 0.53439681, 0.87724551, 0.01696607,
        0.01098901, 0.01495513, 0.0109671 , 0.01696607, 0.22577423]])

8\. Déterminer les paramètres f(0)(b) du modèle nul, où
\begin{equation}
f^{(0)}(b) = \frac 1L \sum_{i=0}^{L-1} \omega_i(b)\ ,
\end{equation}

In [141]:
def f0_calcule(PWM, L):
    """
    Calcul les valeurs de probabilites d'un modele independant de positions (modele Null)
    entrée PWM : matrice de probabilites ou poids positions
    sortie  f_0 : vecteur contenant un modele independant de positions (modele Null)
    """
    return np.sum(PWM,axis=1)/L


f_0 = f0_calcule(PWM, k)
print (f_0)

[0.22186169 0.3836101  0.13691165 0.25761655]


9\. Faites une fonction pour calculer log-rapport de vraisemblancee d'une sequence de taille k et la matrice poids position.

\begin{equation}
\ell(b_0,...,b_{k-1}) = \log_2 \frac {P(b_0,...,b_{k-1} | \omega )
}{P^{(0)}(b_0,...,b_{k-1})}
= \sum_{i=0}^{k-1} \log_2 \frac {\omega_i(b_i)}{f^{(0)}(b_i)}\ .
\end{equation}

In [142]:
def loglikehood(seq, PWM, f_0, L):
	"""
	Calcul le rapport de vraissemblance entre une sequence et une matrice de poids position
	entrée PWM : matrice de probabilites ou poids positions
	entrée f_0 : vecteur contenant le modele independant de positions (modele Null)
	entrée seq : une sequence d'ADN de taille k, ou k est le nombre de colonnes de PWM
	sortie ll : rapport de vraissemblance
	"""
	L_0 = len(seq)
	l_win = 0
	
	for i in range(L) :
		x = 3
		aux = seq[i].lower()
		if aux == 'a' : x = 0
		elif aux == 'c' : x = 1
		elif aux == 'g' : x = 2
	
		l_win += np.log2(PWM[x,i]/f_0[x])

	return l_win

float(loglikehood('acctgcaccc'.upper(), PWM, f_0, k))

3.210041670360414

10\. Faire une fonction que pour chaque séquence cherche la/les positions avec une log-vraisemblance positive.

In [177]:
def searchPWM(sequences, k, PWM, f_0):
    """
    Cherche les positions dans un ensemble de séquence qui maxime le rapport de vraisemblance
    entrée sequences : ensemble de séquence d'ADN
    entrée k : nombre de colonnes d'PWM
    entrée PWM : matrice de probabilités ou poids positions
    entrée f_0 : vecteur contenant le modèle indépendant de positions (modèle Null)
    sortie posList: liste contenant pour chaque séquence la/les positions ayant un rapport de vraisemblance positive
    """
    #count = 0
    posList = []

    all_pos = dict()

    for i in range(len(sequences)) :
        seq = sequences[i]

        pos = []
        for j in range(len(seq)-k+1) :
            if loglikehood(seq[j:j+k],PWM,f_0,k) > 0 : pos.append(j)

        all_pos[i] = pos
        posList += pos

    print(all_pos)

    return posList


posList = searchPWM(sequencesPeaks, k, PWM, f_0)
print(posList)


{0: [15, 30, 120, 363, 401, 427, 444, 533, 620, 660, 668], 1: [59, 95, 182, 526, 598, 783, 841, 946, 994, 1063], 2: [61, 197, 344, 432, 448, 478, 487], 3: [141, 171], 4: [162], 5: [], 6: [], 7: [265, 294], 8: [], 9: [], 10: [90, 123], 11: [30, 139, 288, 463], 12: [70], 13: [74, 89], 14: [48, 122, 135], 15: [], 16: [36, 185], 17: [36, 155, 228, 237, 294, 376], 18: [222, 239, 283], 19: [1], 20: [], 21: [7, 94, 124], 22: [238], 23: [], 24: [144], 25: [], 26: [38], 27: [35, 86], 28: [77, 194, 268, 331], 29: [26, 82, 202], 30: [], 31: [134, 148], 32: [65], 33: [], 34: [], 35: [104], 36: [35], 37: [84], 38: [71, 242, 307, 327, 447, 513], 39: [118, 175, 203, 235, 342, 452], 40: [64, 209, 527, 657, 809, 853, 876, 930, 1079, 1139, 1317, 1479, 1508, 1520, 1559, 1585, 1595, 1732], 41: [88, 131, 284], 42: [95, 193], 43: [], 44: [75, 88, 108], 45: [77, 101], 46: [21, 84, 92, 194, 223, 233, 246, 388], 47: [2, 37, 159, 232], 48: [125, 166, 181, 234, 246, 258], 49: [30, 51, 162], 50: [], 51: [45, 47, 

11\.  (Bonus) Certains motifs si chevauche dans ce cas il faut choisir ceux qui ont la meilleur  log-vraisemblance. Par exemple pour une taille de motif k = 7, si vous avez deux motifs, un qui commence dans la position 45 et un autre en position 47, garder celui avec la log-vraisemblance la plus grande.

In [178]:
def searchPWMOptmiseMotifs(sequences, k, PWM, f_0):
    """
    Cherche les positions dans un ensemble de séquence qui maxime le rapport de vraisemblance et elimine les motifs chevauchante
    entrée sequences : ensemble de séquence d'ADN
    entrée k : nombre de colonnes d'PWM
    entrée PWM : matrice de probabilités ou poids positions
    entrée f_0 : vecteur contenant le modèle indépendant de positions (modèle Null)
    sortie posList: liste contenant pour chaque séquence la/les positions ayant un rapport de vraisemblance positive

    """

    posList = []

    all_pos = dict()


    for i in range(len(sequences)) :
        seq = sequences[i]
        limit = len(seq)

        pos = []
        prev_pos, prev_log = -1,0

        for j in range(len(seq)-k+1) :
            curr_log = loglikehood(seq[j:j+k],PWM,f_0,k)

            if curr_log > 0 :
                if j <= prev_pos+k and curr_log > prev_log : 
                    if prev_pos in pos : pos.remove(prev_pos)
                else : 
                    pos.append(j)
                prev_pos = j
                prev_log = curr_log

        all_pos[i] = pos
        posList += pos
    
    return all_pos

In [210]:
optList = searchPWMOptmiseMotifs(sequencesPeaks, k, PWM, f_0)
print(optList)

{0: [15, 30, 120, 363, 401, 427, 444, 533, 620], 1: [59, 95, 182, 526, 598, 783, 841, 946, 994, 1063], 2: [61, 197, 344, 432, 448], 3: [141, 171], 4: [162], 5: [], 6: [], 7: [265, 294], 8: [], 9: [], 10: [90, 123], 11: [30, 139, 288, 463], 12: [70], 13: [74, 89], 14: [48, 122, 135], 15: [], 16: [36, 185], 17: [36, 155, 228, 237, 294, 376], 18: [222, 239, 283], 19: [], 20: [], 21: [94, 124], 22: [238], 23: [], 24: [144], 25: [], 26: [38], 27: [35, 86], 28: [77, 194, 268, 331], 29: [26, 82, 202], 30: [], 31: [134, 148], 32: [65], 33: [], 34: [], 35: [104], 36: [35], 37: [84], 38: [71, 242, 307, 327, 447, 513], 39: [118, 175, 203, 235, 342, 452], 40: [64, 209, 527, 657, 809, 853, 876, 930, 1079, 1139, 1317, 1479, 1508, 1520, 1559, 1585, 1595, 1732], 41: [88, 131, 284], 42: [95, 193], 43: [], 44: [75, 88, 108], 45: [77, 101], 46: [21, 84, 92, 194, 246, 388], 47: [37, 159, 232], 48: [125, 166, 181, 234, 246, 258], 49: [30, 51, 162], 50: [], 51: [45, 47, 167, 282, 500], 52: [276], 53: [212, 

12\.Tester l'algorithme sur vos données de chipSeq et dessiner le LOGO

In [211]:
peaks1 = searchPWMOptmiseMotifs(sequencesPeaks,7,PWM,f_0) 
print(peaks1)

{0: [30, 71, 91, 144, 155, 176, 238, 264, 288, 325, 347, 363, 375, 380, 392, 401, 427, 444, 455, 481, 496, 519, 533, 541, 561, 620, 624, 635, 660, 662, 683, 688, 812, 814, 840], 1: [37, 55, 71, 97, 116, 131, 158, 182, 217, 222, 234, 245, 254, 255, 283, 307, 353, 366, 380, 386, 390, 398, 406, 411, 429, 467, 513, 526, 571, 583, 585, 598, 641, 650, 703, 751, 758, 783, 799, 843, 902, 907, 912, 932, 946, 956, 966, 975, 1016, 1034, 1038, 1050, 1063, 1078, 1091, 1103], 2: [9, 18, 48, 61, 126, 159, 180, 190, 197, 244, 284, 307, 331, 344, 369, 388, 393, 404, 424, 432, 448, 450, 465, 470, 487, 489], 3: [9, 42, 78, 98, 119, 121, 141, 181, 238], 4: [8, 15, 30, 45, 82, 107, 127, 162, 176, 181, 195, 209, 239, 245, 249, 264, 276], 5: [15, 38, 58, 86, 96, 127, 180, 198, 207, 219], 6: [57, 62, 124, 137, 164, 185], 7: [32, 50, 142, 157, 191, 196, 207, 236, 250, 265, 279, 294, 308, 320], 8: [9, 28, 45, 47, 84, 94, 95, 104, 142], 9: [25, 56, 76, 96, 126, 144], 10: [7, 22, 57, 63, 82, 90, 123], 11: [30, 78

In [212]:
found = dict()
for idx,pos in peaks1.items() :
    seq = sequencesPeaks[idx]
    for p in pos :
        aux = seq[p:p+7]
        if aux in found : found[aux] += 1
        else : found[aux] = 1

In [216]:
from functions import getTopMotifs
getTopMotifs(found,10,True)

{'gggtgca': 17,
 'tatttca': 13,
 'tattgaa': 11,
 'ttattca': 10,
 'aaaaaca': 10,
 'gaaagca': 9,
 'atttgta': 9,
 'taataaa': 9,
 'taaaaca': 8,
 'atttaca': 8}

In [214]:
m1 = 'tgcaccc'
m2 = 'gggtgca'

In [215]:
m1 in found, m2 in found

(False, True)